In [ ]:
from ByBit import ByBitOptionData, recommend_option_position, ByBitAPI


In [ ]:
prod_url  = "https://api.bybit.com"
ticker_endpoint = "/v5/market/tickers"
btc_params = {'category': "option",
              'baseCoin': "BTC"}

daily_delta_limit: float = 0.10  # Daily Delta Deviation Limit
weekly_delta_limit: float = 0.10  # Weekly Delta Deviation Limit
monthly_delta_limit: float = 0.10  # Monthly Delta Deviation Limit
quarterly_delta_limit: float = 0.10  # Quarterly Delta Deviation Limit

In [ ]:
ByBit = ByBitOptionData(api_url = prod_url, api_endpoint = ticker_endpoint, api_parameters = btc_params)

ByBitAPI = ByBitAPI(default_quantity = 1.0,
    api_url = "https://api-demo.bybit.com",
    api_key = "6TzoOeOuyIDiJN9cOu",
    api_secret = "JbTE9Z5OFFyxcw1mzqf6hLIpmGgt8dONmlSE",
    baseCoin = "BTC",
    settleCoin= "USDT")

In [ ]:
await ByBit.fetch_ByBit_ticker_data()
await ByBit.format_the_dataframe()

(current_daily, next_daily, next_to_next_daily,
current_weekly, next_weekly, next_to_next_weekly,
current_monthly, next_monthly, next_to_next_monthly,
current_quarterly, next_quarterly, next_to_next_quarterly, expiry) = await ByBit.segregate_options_expiry()


In [ ]:
json_fut = await ByBitAPI.get_PerpFutures_Position()
json_fut

In [ ]:
len(json_fut.json()["result"]["list"])

In [ ]:
json_data = await ByBitAPI.get_option_positions()
position_df = await ByBitAPI.format_option_position_dataframe()

In [ ]:
async def set_expiry_delta_and_compute_hedging():
    
    expiry["daily"]["delta_limit"] = float(daily_delta_limit)
    expiry["weekly"]["delta_limit"] = float(weekly_delta_limit)
    expiry["monthly"]["delta_limit"] = float(monthly_delta_limit)
    expiry["quarterly"]["delta_limit"] = float(quarterly_delta_limit)

    position_df.loc[position_df.expiry_type == "daily", "expiry_delta"] = daily_delta_limit
    position_df.loc[position_df.expiry_type == "weekly", "expiry_delta"] = weekly_delta_limit
    position_df.loc[position_df.expiry_type == "monthly", "expiry_delta"] = monthly_delta_limit
    position_df.loc[position_df.expiry_type == "quarterly", "expiry_delta"] = quarterly_delta_limit


    for i in range(len(position_df)):
        print(f"Delta Hedging : {abs(round(position_df.loc[i, "expiry_delta"], 6))}",
              f"Expiry Delta {abs(round(position_df.loc[i, "expiry_delta"], 6))}")
        if abs(round(position_df.loc[i, "delta_hedging"], 6)) > abs(round(position_df.loc[i, "expiry_delta"], 6)):
            print(f"Hedging Required !!! ")
            position_df.loc[i, "hedging_required"] = True
            position_df.loc[i, "PerpFutureQty"] = position_df.loc[i, "delta_hedging"]
        else:
            print(f"Hedging Not Required !!! ")
            position_df.loc[i, "hedging_required"] = False
            position_df.loc[i, "PerpFutureQty"] = position_df.loc[i, "delta_hedging"]
    


In [ ]:
await set_expiry_delta_and_compute_hedging()

In [ ]:
async def check_the_position_availablity():
        
    ## Update the Daily Expiry Dictionary 
    # Update the Daily : Current Daily
    if len(position_df.loc[position_df.expiry == expiry["daily"]["current"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["daily"]["current"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["daily"]["current"]["date"], "expiry_type"] = "daily"
        expiry["daily"]["current"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["daily"]["current"]["date"]}")
    
    # Update the Monthly : Next Daily
    if len(position_df.loc[position_df.expiry == expiry["daily"]["next"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["daily"]["next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["daily"]["next"]["date"], "expiry_type"] = "daily"
        expiry["daily"]["next"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["daily"]["next"]["date"]}")
    
    # Update the Monthly : Next_to_Next Daily
    if len(position_df.loc[position_df.expiry == expiry["daily"]["next_to_next"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["daily"]["next_to_next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["daily"]["next_to_next"]["date"], "expiry_type"] = "daily"
        expiry["daily"]["next_to_next"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["daily"]["next_to_next"]["date"]}")
    
    
    
    ## Update the Weekly Expiry Dictionary 
    # Update the Weekly : Current Weekly
    if len(position_df.loc[position_df.expiry == expiry["weekly"]["current"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["weekly"]["current"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["weekly"]["current"]["date"], "expiry_type"] = "weekly"
        expiry["weekly"]["current"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["weekly"]["current"]["date"]}")
    
    # Update the Weekly : Next Weekly
    if len(position_df.loc[position_df.expiry == expiry["weekly"]["next"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["weekly"]["next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["weekly"]["next"]["date"], "expiry_type"] = "weekly"
        expiry["weekly"]["next"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["weekly"]["next"]["date"]}")
    
    # Update the Weekly : Next_to_Next Weekly
    if len(position_df.loc[position_df.expiry == expiry["weekly"]["next_to_next"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["weekly"]["next_to_next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["weekly"]["next_to_next"]["date"], "expiry_type"] = "weekly"
        expiry["weekly"]["next_to_next"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["weekly"]["next_to_next"]["date"]}")
    
    ## Update the Monthly Expiry Dictionary 
    # Update the Monthly : Current Monthly
    if len(position_df.loc[position_df.expiry == expiry["monthly"]["current"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["monthly"]["current"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["monthly"]["current"]["date"], "expiry_type"] = "monthly"
        expiry["monthly"]["current"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["monthly"]["current"]["date"]}")
    
    # Update the Monthly : Next Monthly
    if len(position_df.loc[position_df.expiry == expiry["monthly"]["next"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["monthly"]["next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["monthly"]["next"]["date"], "expiry_type"] = "monthly"
        expiry["monthly"]["next"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["monthly"]["next"]["date"]}")
    
    # Update the Monthly : Next_to_Next Monthly
    if len(position_df.loc[position_df.expiry == expiry["monthly"]["next_to_next"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["monthly"]["next_to_next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["monthly"]["next_to_next"]["date"], "expiry_type"] = "monthly"
        expiry["monthly"]["next_to_next"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["monthly"]["next_to_next"]["date"]}")
    
    
    
    ## Update the Quarterly Expiry Dictionary 
    # Update the Quarterly : Current Quarterly
    if len(position_df.loc[position_df.expiry == expiry["quarterly"]["current"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["quarterly"]["current"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["quarterly"]["current"]["date"], "expiry_type"] = "quarterly"
        expiry["quarterly"]["current"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["quarterly"]["current"]["date"]}")
    
    # Update the Quarterly : Next Quarterly
    if len(position_df.loc[position_df.expiry == expiry["quarterly"]["next"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["quarterly"]["next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["quarterly"]["next"]["date"], "expiry_type"] = "quarterly"
        expiry["quarterly"]["next"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["quarterly"]["next"]["date"]}")
    
    # Update the Quarterly : Next_to_Next Quarterly
    if len(position_df.loc[position_df.expiry == expiry["quarterly"]["next_to_next"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["quarterly"]["next_to_next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["quarterly"]["next_to_next"]["date"], "expiry_type"] = "quarterly"
        expiry["quarterly"]["next_to_next"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["quarterly"]["next_to_next"]["date"]}")

In [ ]:
await check_the_position_availablity()

In [ ]:
position_df

In [ ]:
current_daily_call, current_daily_put = await recommend_option_position(option_chain=current_daily)
next_daily_call, next_daily_put = await recommend_option_position(option_chain=next_daily)
next_to_next_daily_call, next_to_next_daily_put = await recommend_option_position(option_chain=next_to_next_daily)

current_weekly_call, current_weekly_put = await recommend_option_position(option_chain=current_weekly)
next_weekly_call, next_weekly_put = await recommend_option_position(option_chain=next_weekly)
next_to_next_weekly_call, next_to_next_weekly_put = await recommend_option_position(option_chain=next_to_next_weekly)

current_monthly_call, current_monthly_put = await recommend_option_position(option_chain=current_monthly)
next_monthly_call, next_monthly_put = await recommend_option_position(option_chain=next_monthly)
next_to_next_monthly_call, next_to_next_monthly_put = await recommend_option_position(option_chain=next_to_next_monthly)

current_quarterly_call, current_quarterly_put = await recommend_option_position(option_chain=current_quarterly)
next_quarterly_call, next_quarterly_put = await recommend_option_position(option_chain=next_quarterly)
next_to_next_quarterly_call, next_to_next_quarterly_put = await recommend_option_position(option_chain=next_to_next_quarterly)

In [ ]:
expiry.keys()

In [ ]:
if current_daily_call is not None and current_daily_put is not None and expiry["daily"]["current"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_daily_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_daily_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Daily Option as NO Matching Criteria Exist")

if next_daily_call is not None and next_daily_put is not None and expiry["daily"]["next"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_daily_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_daily_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Next Daily Option as NO Matching Criteria Exist")

if next_to_next_daily_call is not None and next_to_next_daily_put is not None and expiry["daily"]["next_to_next"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_daily_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_daily_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Next to Next Daily Option as NO Matching Criteria Exist")

############
if current_weekly_call is not None and current_weekly_put is not None and expiry["weekly"]["current"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_weekly_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_weekly_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Weekly Option as NO Matching Criteria Exist")


if next_weekly_call is not None and next_weekly_put is not None and expiry["weekly"]["next"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_weekly_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_weekly_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Next Weekly Option as NO Matching Criteria Exist")

if next_to_next_weekly_call is not None and next_to_next_weekly_put is not None and expiry["weekly"]["next_to_next"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_weekly_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_weekly_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Next to Next Weekly Option as NO Matching Criteria Exist")


############
if current_monthly_call is not None and current_monthly_put is not None and expiry["monthly"]["current"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_monthly_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_monthly_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Monthly Option as NO Matching Criteria Exist")

if next_monthly_call is not None and next_monthly_put is not None and expiry["monthly"]["next"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_monthly_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_monthly_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Next Monthly Option as NO Matching Criteria Exist")

if next_to_next_monthly_call is not None and next_to_next_monthly_put is not None and expiry["monthly"]["next_to_next"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_monthly_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_monthly_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Next to Next Monthly Option as NO Matching Criteria Exist")

############
if current_quarterly_call is not None and current_quarterly_put is not None and expiry["quarterly"]["current"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_quarterly_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_quarterly_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Quarterly Option as NO Matching Criteria Exist")

if next_quarterly_call is not None and next_quarterly_put is not None and expiry["quarterly"]["next"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_quarterly_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_quarterly_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Next Quarterly Option as NO Matching Criteria Exist")

if next_to_next_quarterly_call is not None and next_to_next_quarterly_put is not None and expiry["quarterly"]["next_to_next"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_quarterly_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_quarterly_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Next to Next Quarterly Option as NO Matching Criteria Exist")


In [ ]:
from ByBit import ByBitAPI

In [ ]:
position_df.info()

In [ ]:
position_df

In [ ]:
expiry_dataframe = position_df.drop(columns=["symbol", "avgPrice", "delta", "theta", "positionValue", "unrealisedPnl", "size", "markPrice",
                            "createdTime", "seq", "updatedTime","side","curRealisedPnl","positionStatus", "gamma",
                            "vega", "option_type","strike_price" ]).copy()

In [ ]:
async def check_delta_computation():
    if round(sum(position_df.total_delta), 6) == round(sum(position_df.delta_hedging)/2, 6):
        print(f"Total Delta is Computed Correctly !!! , Total Options Open Positional Detla is : ",
              f"{sum(position_df.total_delta)}",
              f"Total Delta Computed for Hedging : {sum(position_df.delta_hedging)/2}")
    
    else:
        print(f"ERROR !!!! Delta Computed In-Correctly",
              f"Total Delta : {sum(position_df.total_delta)}",
              f"Total Delta Computed for Hedging : {sum(position_df.delta_hedging)/2}")

In [ ]:
await check_delta_computation()

In [ ]:
expiry_dataframe

In [ ]:
await check_the_position_availablity()

In [ ]:
expiry

In [1]:
from ByBit import CoreLogic

In [2]:
btcCoreLogic = CoreLogic(
    api_url = "https://api-demo.bybit.com",  # API URL
    api_endpoint = "/v5/market/tickers",  # Endpoint URI
    api_keys = "6TzoOeOuyIDiJN9cOu",  # API Keys
    api_secret = "JbTE9Z5OFFyxcw1mzqf6hLIpmGgt8dONmlSE",  # API Secret
    baseCoin = "BTC",  # BaseCoin neets be either BTC, ETH, SOL Only
    settleCoin = "USDC",
    default_quantity = 1.00,  # Default Short Quantity

    daily = True,  # Play on Daily Options: True or False
    weekly = True,  # Play on Weekly Options: True or False
    monthly = True,  # Play on Monthly Options: True or False
    quarterly = True,  # Play on Quarterly Options: True or False

    delta_value = 0.07,  # Delta Value beyond which we would take our Positions
    min_bid_price = 0.01,  # Minumum Bid Price to eliminate the Blank Bid Strikes
    initial_mark_price_diff = 0.05,  # The Difference Between Mark & Bid Price 0.05 means 5%
    max_mark_price_diff = 0.3,  # Maximum Difference between Mark & Bid Price
    mark_price_diff_steps = 0.05,  # Mark & Ask Difference Percentage Incremental Step Size
    
    # Tolarated Delta Deavation Limit, after which we Negate the Delta Deavation
    # by hedging with Perpectual Futuers
    daily_delta_limit = 0.10,  # Daily Delta Deviation Limit
    weekly_delta_limit = 0.10,  # Weekly Delta Deviation Limit
    monthly_delta_limit = 0.10,  # Monthly Delta Deviation Limit
    quarterly_delta_limit = 0.10,  # Quarterly Delta Deviation Limit
)

In [3]:
await btcCoreLogic.core_logic_computation()

Class Variables Reinitialization Initiated
Tickers Data Retrieval Success : Data Count : 516
Data Fetching from ByBit API is Completed in : 784.0260160010075 ms 

Formatting & Processing of DataFrame is Completed in : 81.48621399595868 ms 

2024-10-27
2024-10-28
2024-10-29
2024-11-01
2024-11-08
2024-11-15
2024-11-29
2024-12-27
2025-03-28
2025-06-27
2025-09-26
Initial Current Month is  11
Original Length of Weekly Expiry List Array : 67, But Length of returned Array : 5 
Original Length of Monthly Expiry List Array : 20, But Length of returned Array : 4 
Original Length of Weekly Expiry List Array : 6, But Length of returned Array : 5 
Setting expiry[weekly][current][date] as : 2024-11-01 Setting expiry[weekly][current][exist] as : True
Setting expiry[weekly][next][date] as : 2024-11-08 Setting expiry[weekly][next][exist] as : True
Setting expiry[weekly][next_to_next][date] as : 2024-11-15 Setting expiry[weekly][next_to_next][exist] as : True
Setting expiry[monthly][current][date] as : 

In [5]:
btcCoreLogic.position_df.sort_values("expiry", inplace=True, ignore_index=True)

In [8]:
btcCoreLogic.position_df.markPrice = round(btcCoreLogic.position_df.markPrice, 6)
btcCoreLogic.position_df.delta = round(btcCoreLogic.position_df.delta, 6)
btcCoreLogic.position_df.theta = round(btcCoreLogic.position_df.theta, 6)
btcCoreLogic.position_df.positionValue = round(btcCoreLogic.position_df.positionValue, 6)

btcCoreLogic.position_df.gamma = round(btcCoreLogic.position_df.gamma, 6)
btcCoreLogic.position_df.vega = round(btcCoreLogic.position_df.vega, 6)
btcCoreLogic.position_df.total_delta = round(btcCoreLogic.position_df.total_delta, 6)
btcCoreLogic.position_df.expiry_delta = round(btcCoreLogic.position_df.expiry_delta, 6)
btcCoreLogic.position_df.delta_hedging = round(btcCoreLogic.position_df.delta_hedging, 6)
btcCoreLogic.position_df.PerpFutureQty = round(btcCoreLogic.position_df.PerpFutureQty, 6)


In [9]:
btcCoreLogic.position_df

,symbol,avgPrice,delta,theta,positionValue,unrealisedPnl,markPrice,createdTime,seq,updatedTime,...,vega,option_type,strike_price,expiry,total_delta,expiry_delta,expiry_type,delta_hedging,hedging_required,PerpFutureQty
0,BTC-27OCT24-72000-C,35.0,-0.000000,0.000000,-0.000000,35.000000,0.000000,2024-10-24 17:45:02.610,671306216,2024-10-24 17:45:02.610,...,-0.000000,CALL,72000.0,2024-10-27,-0.000000,0.1,daily,-0.000000,False,-0.000000
1,BTC-27OCT24-64000-P,35.0,0.000000,0.000000,0.000000,35.000000,0.000000,2024-10-24 17:45:02.810,671306217,2024-10-24 17:45:02.810,...,-0.000000,PUT,64000.0,2024-10-27,0.000000,0.1,daily,-0.000000,False,-0.000000
2,BTC-28OCT24-72000-C,25.0,-0.001823,4.589633,-0.845002,24.154998,0.845002,2024-10-25 12:35:02.479,671313655,2024-10-25 12:35:02.480,...,-0.207032,CALL,72000.0,2024-10-28,-0.001823,0.1,daily,0.050764,False,0.050764
3,BTC-28OCT24-65000-P,30.0,0.052587,70.643913,-30.027110,-0.027110,30.027110,2024-10-25 12:35:02.855,671313656,2024-10-25 12:35:02.856,...,-3.813940,PUT,65000.0,2024-10-28,0.052587,0.1,daily,0.050764,False,0.050764
4,BTC-29OCT24-64000-P,55.0,0.061599,63.589392,-57.041275,-2.041275,57.041275,2024-10-27 04:14:30.357,671328974,2024-10-27 04:14:30.358,...,-6.077679,PUT,64000.0,2024-10-29,0.061599,0.1,daily,0.037585,False,0.037585
5,BTC-29OCT24-71000-C,20.0,-0.024014,26.264874,-16.808724,3.191276,16.808724,2024-10-27 04:14:30.163,671328973,2024-10-27 04:14:30.163,...,-2.824168,CALL,71000.0,2024-10-29,-0.024014,0.1,daily,0.037585,False,0.037585
6,BTC-1NOV24-58000-P,195.0,0.019550,23.101315,-35.839748,159.160252,35.839748,2024-10-18 06:00:37.377,671276189,2024-10-18 06:00:37.378,...,-3.743989,PUT,58000.0,2024-11-01,0.019550,0.1,weekly,0.007540,False,0.007540
7,BTC-1NOV24-82000-C,125.0,-0.012010,18.088752,-23.488287,101.511713,23.488287,2024-10-18 06:00:37.171,671276188,2024-10-18 06:00:37.172,...,-2.464090,CALL,82000.0,2024-11-01,-0.012010,0.1,weekly,0.007540,False,0.007540
8,BTC-8NOV24-54000-P,290.0,0.039699,31.365292,-147.933833,142.066167,147.933833,2024-10-18 06:00:38.007,671276191,2024-10-18 06:00:38.007,...,-10.467567,PUT,54000.0,2024-11-08,0.039699,0.1,weekly,0.018617,False,0.018617
9,BTC-8NOV24-88000-C,215.0,-0.021082,18.101159,-64.217738,150.782262,64.217738,2024-10-18 06:00:37.732,671276190,2024-10-18 06:00:37.732,...,-6.187656,CALL,88000.0,2024-11-08,-0.021082,0.1,weekly,0.018617,False,0.018617


In [10]:
expiry_df = btcCoreLogic.position_df.drop(columns=["symbol", "avgPrice", "delta", "theta", "positionValue", "unrealisedPnl", "size", "markPrice",
                            "createdTime", "seq", "updatedTime","side","curRealisedPnl","positionStatus", "gamma",
                            "vega", "option_type","strike_price" ]).copy()

In [11]:
expiry_df.drop_duplicates("expiry", inplace=True, ignore_index=True)

In [12]:
expiry_df

,expiry,total_delta,expiry_delta,expiry_type,delta_hedging,hedging_required,PerpFutureQty
0,2024-10-27,-0.000000,0.1,daily,-0.000000,False,-0.000000
1,2024-10-28,-0.001823,0.1,daily,0.050764,False,0.050764
2,2024-10-29,0.061599,0.1,daily,0.037585,False,0.037585
3,2024-11-01,0.019550,0.1,weekly,0.007540,False,0.007540
4,2024-11-08,0.039699,0.1,weekly,0.018617,False,0.018617
5,2024-11-15,-0.070286,0.1,weekly,-0.021044,False,-0.021044
6,2024-11-29,0.057978,0.1,monthly,0.019287,False,0.019287
7,2024-12-27,0.018560,0.1,monthly,-0.001222,False,-0.001222
8,2025-03-28,0.012156,0.1,quarterly,-0.006169,False,-0.006169
9,2025-09-26,-0.022464,0.1,quarterly,-0.016920,False,-0.016920


In [13]:
btcCoreLogic.position_df.loc[btcCoreLogic.position_df.expiry == expiry_df.loc[0,"expiry"]]

,symbol,avgPrice,delta,theta,positionValue,unrealisedPnl,markPrice,createdTime,seq,updatedTime,...,vega,option_type,strike_price,expiry,total_delta,expiry_delta,expiry_type,delta_hedging,hedging_required,PerpFutureQty
0,BTC-27OCT24-72000-C,35.0,-0.0,0.0,-0.0,35.0,0.0,2024-10-24 17:45:02.610,671306216,2024-10-24 17:45:02.610,...,-0.0,CALL,72000.0,2024-10-27,-0.0,0.1,daily,-0.0,False,-0.0
1,BTC-27OCT24-64000-P,35.0,0.0,0.0,0.0,35.0,0.0,2024-10-24 17:45:02.810,671306217,2024-10-24 17:45:02.810,...,-0.0,PUT,64000.0,2024-10-27,0.0,0.1,daily,-0.0,False,-0.0


In [14]:
import numpy as np 
avgPrice = np.array([-5000.00 for _ in range(len(expiry_df))])
markPrice = np.array([-5000.00 for _ in range(len(expiry_df))])
unrealisedPnl = np.array([-5000.00 for _ in range(len(expiry_df))])

expiry_df["avgPrice"] = avgPrice
expiry_df["markPrice"] = markPrice
expiry_df["unrealisedPnl"] = unrealisedPnl


In [15]:
for i in range(len(expiry_df)):
    expiry_df.loc[i, "markPrice"] = sum(btcCoreLogic.position_df.loc[
                                        btcCoreLogic.position_df.expiry == expiry_df.loc[i , "expiry"], "markPrice"])
    expiry_df.loc[i, "avgPrice"] = sum(btcCoreLogic.position_df.loc[
                                       btcCoreLogic.position_df.expiry == expiry_df.loc[i ,"expiry"], "avgPrice"])
    expiry_df.loc[i, "unrealisedPnl"] = sum(btcCoreLogic.position_df.loc[
                                            btcCoreLogic.position_df.expiry == expiry_df.loc[i , "expiry"], "unrealisedPnl"])
    

In [16]:
expiry_df

,expiry,total_delta,expiry_delta,expiry_type,delta_hedging,hedging_required,PerpFutureQty,avgPrice,markPrice,unrealisedPnl
0,2024-10-27,-0.000000,0.1,daily,-0.000000,False,-0.000000,70.0,0.000000,70.000000
1,2024-10-28,-0.001823,0.1,daily,0.050764,False,0.050764,55.0,30.872112,24.127888
2,2024-10-29,0.061599,0.1,daily,0.037585,False,0.037585,75.0,73.849999,1.150001
3,2024-11-01,0.019550,0.1,weekly,0.007540,False,0.007540,320.0,59.328035,260.671965
4,2024-11-08,0.039699,0.1,weekly,0.018617,False,0.018617,505.0,212.151571,292.848429
5,2024-11-15,-0.070286,0.1,weekly,-0.021044,False,-0.021044,480.0,487.747720,-7.747720
6,2024-11-29,0.057978,0.1,monthly,0.019287,False,0.019287,760.0,515.483776,244.516225
7,2024-12-27,0.018560,0.1,monthly,-0.001222,False,-0.001222,410.0,61.063402,697.873195
8,2025-03-28,0.012156,0.1,quarterly,-0.006169,False,-0.006169,465.0,322.908984,142.091015
9,2025-09-26,-0.022464,0.1,quarterly,-0.016920,False,-0.016920,155.0,399.302084,-244.302084


In [17]:
delta_risk_multiplier:float = 2.0
total_perpFutures:float = 0.0
temp_count:float = 0.0
total_delta_risk_magnitude:float = 0.0


total_perpFutures = round(sum(expiry_df.loc[expiry_df.hedging_required == True, "PerpFutureQty"]), 6)
total_perpFutures

0

In [18]:
daily_delta_risk = round(sum(expiry_df[expiry_df.expiry_type == "daily"].PerpFutureQty), 6)
daily_delta_risk_magnitude = delta_risk_multiplier * 0.1

if daily_delta_risk > daily_delta_risk_magnitude:
    
    total_delta_risk_magnitude = total_delta_risk_magnitude + daily_delta_risk
    
    print(f"Attention !!!! : Daily Delta Risk Paramater Exceeded : {daily_delta_risk_magnitude}, "
    f"Risk Magnitude : {daily_delta_risk}, Total Risk Magnitude: {total_delta_risk_magnitude}")

else:
    print(f"Daily Delta Risk Paramater is within Limit : {daily_delta_risk_magnitude} ",
         f"Risk Magnitude : {daily_delta_risk}, Total Risk Magnitude: {total_delta_risk_magnitude}")

Daily Delta Risk Paramater is within Limit : 0.2  Risk Magnitude : 0.088349, Total Risk Magnitude: 0.0


In [19]:
expiry_df[expiry_df.expiry_type == "weekly"]

,expiry,total_delta,expiry_delta,expiry_type,delta_hedging,hedging_required,PerpFutureQty,avgPrice,markPrice,unrealisedPnl
3,2024-11-01,0.019550,0.1,weekly,0.007540,False,0.007540,320.0,59.328035,260.671965
4,2024-11-08,0.039699,0.1,weekly,0.018617,False,0.018617,505.0,212.151571,292.848429
5,2024-11-15,-0.070286,0.1,weekly,-0.021044,False,-0.021044,480.0,487.747720,-7.747720


In [20]:
weekly_delta_risk = round(sum(expiry_df[expiry_df.expiry_type == "weekly"].PerpFutureQty), 6)
weekly_delta_risk_magnitude = delta_risk_multiplier * 0.1

if weekly_delta_risk > weekly_delta_risk_magnitude:
    
    total_delta_risk_magnitude = total_delta_risk_magnitude + weekly_delta_risk

    print(f"Attention !!!! : Weekly Delta Risk Paramater Exceeded : {weekly_delta_risk}, "
    f"Risk Magnitude : {weekly_delta_risk_magnitude}, Total Risk Magnitude: {total_delta_risk_magnitude}")

else:
    print(f"Weekly Delta Risk Paramater is within Limit : {weekly_delta_risk} ",
         f"Risk Magnitude : {weekly_delta_risk_magnitude}, Total Risk Magnitude: {total_delta_risk_magnitude}")

Weekly Delta Risk Paramater is within Limit : 0.005113  Risk Magnitude : 0.2, Total Risk Magnitude: 0.0


In [21]:
expiry_df[expiry_df.expiry_type == "monthly"]

,expiry,total_delta,expiry_delta,expiry_type,delta_hedging,hedging_required,PerpFutureQty,avgPrice,markPrice,unrealisedPnl
6,2024-11-29,0.057978,0.1,monthly,0.019287,False,0.019287,760.0,515.483776,244.516225
7,2024-12-27,0.018560,0.1,monthly,-0.001222,False,-0.001222,410.0,61.063402,697.873195


In [22]:
monthly_delta_risk = round(sum(expiry_df[expiry_df.expiry_type == "monthly"].PerpFutureQty), 6)
monthly_delta_risk_magnitude = delta_risk_multiplier * 0.1

if monthly_delta_risk > monthly_delta_risk_magnitude:
    total_delta_risk_magnitude = total_delta_risk_magnitude + monthly_delta_risk

    print(f"Attention !!!! : Monthly Delta Risk Paramater Exceeded : {monthly_delta_risk}, "
    f"Risk Magnitude : {monthly_delta_risk_magnitude}, Total Risk Magnitude: {total_delta_risk_magnitude}")

else:
    print(f"Monthly Delta Risk Paramater is within Limit : {monthly_delta_risk} ",
         f"Risk Magnitude : {monthly_delta_risk_magnitude}, Total Risk Magnitude: {total_delta_risk_magnitude}")

Monthly Delta Risk Paramater is within Limit : 0.018065  Risk Magnitude : 0.2, Total Risk Magnitude: 0.0


In [23]:
expiry_df[expiry_df.expiry_type == "quarterly"]

,expiry,total_delta,expiry_delta,expiry_type,delta_hedging,hedging_required,PerpFutureQty,avgPrice,markPrice,unrealisedPnl
8,2025-03-28,0.012156,0.1,quarterly,-0.006169,False,-0.006169,465.0,322.908984,142.091015
9,2025-09-26,-0.022464,0.1,quarterly,-0.016920,False,-0.016920,155.0,399.302084,-244.302084


In [24]:
quarterly_delta_risk = round(sum(expiry_df[expiry_df.expiry_type == "quarterly"].PerpFutureQty), 6)
quarterly_delta_risk_magnitude = delta_risk_multiplier * 0.1

if quarterly_delta_risk > quarterly_delta_risk_magnitude:
    
    total_delta_risk_magnitude = total_delta_risk_magnitude + quarterly_delta_risk
    
    print(f"Attention !!!! : Quarterly Delta Risk Paramater Exceeded : {quarterly_delta_risk}, "
    f"Risk Magnitude : {quarterly_delta_risk_magnitude}, Total Risk Magnitude: {total_delta_risk_magnitude}")

else:
    print(f"Quarterly Delta Risk Paramater is within Limit : {quarterly_delta_risk} ",
         f"Risk Magnitude : {quarterly_delta_risk_magnitude}, Total Risk Magnitude: {total_delta_risk_magnitude}")

Quarterly Delta Risk Paramater is within Limit : -0.023089  Risk Magnitude : 0.2, Total Risk Magnitude: 0.0


In [25]:
sum((btcCoreLogic.position_df["size"] * btcCoreLogic.position_df["avgPrice"]) - (btcCoreLogic.position_df["markPrice"] * btcCoreLogic.position_df["size"]))


1481.2289150000001

In [26]:
sum((btcCoreLogic.position_df["size"] * btcCoreLogic.position_df["avgPrice"]) - (btcCoreLogic.position_df["markPrice"] * btcCoreLogic.position_df["size"]))


1481.2289150000001

In [27]:
((total_perpFutures >= total_delta_risk_magnitude) and total_perpFutures != 0.0), total_perpFutures >= total_delta_risk_magnitude 


(False, True)

In [28]:
total_delta_risk_magnitude, total_perpFutures, 


(0.0, 0)

In [29]:
from ByBit import ByBitAPI
baseCoin = "BTC"
ByBitAPI = ByBitAPI(default_quantity = 1.0,
                    api_url = "https://api-demo.bybit.com",
                    api_key = "6TzoOeOuyIDiJN9cOu",
                    api_secret = "JbTE9Z5OFFyxcw1mzqf6hLIpmGgt8dONmlSE",
                    baseCoin = "BTC",                 
                    settleCoin = "USDC")

In [30]:
try :
    PerpFutures_json_data = await ByBitAPI.get_PerpFutures_Position()
    if len(PerpFutures_json_data.json()["result"]["list"]) > 0:
        PerpFutures_df = await ByBitAPI.format_perpfutures()
        
        if PerpFutures_df.loc[0, "side"] == "BUY":
            PerpFutures_position = float(PerpFutures_df.loc[0,"size"])
        elif PerpFutures_df.loc[0, "side"] == "SELL":
            PerpFutures_position = float(PerpFutures_df.loc[0,"size"]) * (-1.0)
    
        print(f"Existing Perpetual Futures Position Size : {PerpFutures_position}")
        
    else:
        PerpFutures_position = 0.0
        print(f"No {baseCoin}, Perpectual Futires position Exists, Setting Value for 'PerpFutures_position'  :  {PerpFutures_position}")
except Exception as e:
    print(f"Error in Fetcting the Perpectual Futures Position Error Details : {e}")

Parameter String is : 17300148853246TzoOeOuyIDiJN9cOu5000category=linear&baseCoin=BTC&settleCoin=USDC

Generated Signature is : fe0036fc7aa36bb8d05f7ef17e72af81506b40c0b1ad818d156a8bf657137b25

Perpectual Futures Position Info API Signature : fe0036fc7aa36bb8d05f7ef17e72af81506b40c0b1ad818d156a8bf657137b25

API Header is : {'X-BAPI-API-KEY': '6TzoOeOuyIDiJN9cOu', 'X-BAPI-SIGN': 'fe0036fc7aa36bb8d05f7ef17e72af81506b40c0b1ad818d156a8bf657137b25', 'X-BAPI-SIGN-TYPE': '2', 'X-BAPI-TIMESTAMP': '1730014885324', 'X-BAPI-RECV-WINDOW': '5000', 'Content-Type': 'application/json'} 

Final URL is : https://api-demo.bybit.com/v5/position/list?category=linear&baseCoin=BTC&settleCoin=USDC
Positions Info Length is : 0
No BTC, Perpectual Futires position Exists, Setting Value for 'PerpFutures_position'  :  0.0


In [33]:
PerpFutures_position, total_perpFutures, total_delta_risk_magnitude

(0.0, 0, 0.0)

In [32]:
if (total_perpFutures >= total_delta_risk_magnitude):
    # Take the Perpectual Futuers Position
    required_PerfFutures_position = total_perpFutures - PerpFutures_position
    
    if required_PerfFutures_position > 0.0:
        print(f"In Main If Block required_PerfFutures_position > 0.0 , : {required_PerfFutures_position} > 0, ",  
              f"PerpFutures_position : {PerpFutures_position}")
        # await ByBitAPI.create_PerpFutures_Order(direction="BUY", quantity=abs(required_PerfFutures_position))
    elif required_PerfFutures_position < 0.0:
        print(f"ELIf Block required_PerfFutures_position < 0.0 , : {required_PerfFutures_position} < 0, ",  
              f"PerpFutures_position : {PerpFutures_position}")
        # await ByBitAPI.create_PerpFutures_Order(direction="SELL", quantity=abs(required_PerfFutures_position))

elif (total_delta_risk_magnitude >= total_perpFutures):
    # Take the Perpectual Futuers Position
    required_PerfFutures_position = total_delta_risk_magnitude - PerpFutures_position
    
    if required_PerfFutures_position > 0.0:
        # await ByBitAPI.create_PerpFutures_Order(direction="BUY", quantity=abs(required_PerfFutures_position))
        print(f"If Block required_PerfFutures_position > 0.0 , : {required_PerfFutures_position} > 0, ",  
              f"PerpFutures_position : {PerpFutures_position}")
    elif required_PerfFutures_position < 0.0:
        print(f"If Block required_PerfFutures_position > 0.0 , : {required_PerfFutures_position} > 0, ",  
              f"PerpFutures_position : {PerpFutures_position}")
        # await ByBitAPI.create_PerpFutures_Order(direction="SELL", quantity=abs(required_PerfFutures_position))

In [ ]:
total_perpFutures - PerpFutures_position, total_delta_risk_magnitude - PerpFutures_position


In [ ]:
abs(required_PerfFutures_position)

In [ ]:
sum(expiry_df.loc[expiry_df.hedging_required == True, "PerpFutureQty"])

In [ ]:
time.sleep(10)

In [ ]:
await asyncio.sleep(5)

In [ ]:
import datetime 
import time
import time
import asyncio

In [ ]:
utcnow = datetime.datetime.now(datetime.UTC).
utcnow

In [ ]:
if utcnow.minute % 5 == 0 and utcnow.second == 0:
    print(executing Function Current time {str(utcnow)})

else:
    await asyncio.sleep( )

In [ ]:
str(utcnow), (5 - (datetime.datetime.now(datetime.UTC).minute % 5)) * 60, datetime.datetime.now(datetime.UTC).second 


In [ ]:
(5 - (datetime.datetime.now(datetime.UTC).minute % 5)) * 60 + datetime.datetime.now(datetime.UTC).second 


In [ ]:
i = 0
while i <= 3:
    current_utc_time = datetime.datetime.now(datetime.UTC)
    if (current_utc_time.minute % 5 ) == 0:
        print(f"Executimg Function {str(current_utc_time)},"
              f"Wait Timer : {(4 - (current_utc_time.minute % 5)) * 60 + (60 - current_utc_time.second)}")
        await asyncio.sleep(60)
    else:
        print(f"Entering Sleep at {str(current_utc_time)}",
             f"Wait Timer : {(4 - (current_utc_time.minute % 5)) * 60 + (60 - current_utc_time.second)}")
        await asyncio.sleep((4 - (current_utc_time.minute % 5)) * 60 + (60 - current_utc_time.second))
    i = i + 1

In [ ]:
current_utc_time = datetime.datetime.now(datetime.UTC)

(4 - (current_utc_time.minute % 5)) * 60 + (60 - current_utc_time.second), current_utc_time.minute, current_utc_time.second

In [ ]:
current_utc_time

In [ ]:
import datetime
datetime.datetime.now(datetime.UTC)